In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/alpaca-babsa/absa_alpaca.jsonl
/kaggle/input/banglabook-test-set/test_set.csv


In [4]:
!pip install "unsloth[kaggle-new] @ git+https://github.com/unslothai/unsloth.git"

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-98y8lsxz/unsloth_d1a3074bdb70494fbe02018c0f8d97f9
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-98y8lsxz/unsloth_d1a3074bdb70494fbe02018c0f8d97f9
  Resolved https://github.com/unslothai/unsloth.git to commit 229e2ecc67756f36316dfcbea42396f59eef44e0
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of trl to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 30.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 563.4/563.4 kB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00:00:0100

In [5]:
import torch
from unsloth import FastLanguageModel

# Model configuration
model_name = "unsloth/llama-3-8b-bnb-4bit"
max_seq_length = 2048
dtype = None
load_in_4bit = True

# Load the model using Unsloth's FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

# ===================================================================
# THE FIX: Add a padding token to the tokenizer.
# Llama 3 models do not have a pad token by default, which can cause issues
# with batching. We'll use the EOS token as the pad token.
# ===================================================================
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-09-21 20:19:44.856300: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758485985.083242      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758485985.139097      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.9.7: Fast Llama patching. Transformers: 4.55.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

In [6]:
from datasets import load_dataset

# This is the standard Alpaca prompt template
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

# We need to add the EOS (End of Sequence) token to the end of the prompt
# The tokenizer will handle tokenizing the entire string.
EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"]
    texts = []
    for instruction, input_text, output in zip(instructions, inputs, outputs):
        # Format the prompt and add the EOS token
        text = alpaca_prompt.format(instruction, input_text, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

# Load your dataset from the file you uploaded to Kaggle
# The path will be something like '/kaggle/input/your-dataset-name/your-file.json'
dataset_path = "/kaggle/input/alpaca-babsa/absa_alpaca.jsonl"  # <--- CHANGE THIS
dataset = load_dataset("json", data_files=dataset_path, split="train")

# Apply the formatting function to your dataset
dataset = dataset.map(
    formatting_prompts_func, 
    batched = True, 
    remove_columns = ["instruction", "input", "output"] # This is the crucial line
)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/28033 [00:00<?, ? examples/s]

In [7]:
dataset

Dataset({
    features: ['text'],
    num_rows: 28033
})

In [8]:
# Add LoRA adapters to the model
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.9.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [9]:
from trl import SFTConfig, SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    packing = False, # Can make training 5x faster for short sequences.
    args = SFTConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)


Unsloth: Tokenizing ["text"] (num_proc=8):   0%|          | 0/28033 [00:00<?, ? examples/s]

In [10]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
6.863 GB of memory reserved.


In [ ]:
# Start the training process
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 28,033 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.224100
2,1.187600
3,1.130300
4,1.040800
5,0.925500
6,0.862100
7,0.756500
8,0.631500
9,0.519200
10,0.566400


In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

In [ ]:
import pandas as pd
from tqdm import tqdm
import re

# Your fixed instruction
instruction = "নীচের রিভিউ থেকে সব অ্যাসপেক্ট শনাক্ত করুন এবং প্রতিটির অনুভূতি (Positive, Negative, Neutral) নির্ধারণ করুন। আউটপুট JSON array হিসেবে দিন, প্রতিটি অবজেক্টে 'aspect' এবং 'sentiment' কী থাকবে।"

# Make sure the model is ready for inference
FastLanguageModel.for_inference(model)

def generate_for_review(review: str) -> str:
    try:
        # Format alpaca style prompt
        prompt = alpaca_prompt.format(
            instruction,
            review,
            ""  # leave output blank for generation
        )

        # Tokenize
        inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

        # Generate
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=128,
                use_cache=True
            )

        # Decode
        decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

        # ---- CLEANING STEP ----
        # Remove the prompt text from the output (keep only model's answer)
        cleaned = decoded.replace(prompt, "").strip()

        # If JSON array is expected, try to extract it
        match = re.search(r"\[.*\]", cleaned, re.DOTALL)
        if match:
            cleaned = match.group(0).strip()

        return cleaned if cleaned else decoded.strip()
    except Exception as e:
        return f"Error: {str(e)}"

df = pd.read_csv("/kaggle/input/banglabook-test-set/test_set.csv")

# Apply row by row with tqdm progress bar
tqdm.pandas()
df["generation"] = df["review"].progress_apply(generate_for_review)

# Show result
print(df.head())